In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train = pd.read_csv("/kaggle/input/titanic/train.csv")
df_train


In [ ]:
df_test = pd.read_csv("/kaggle/input/titanic/test.csv")
df_test

In [ ]:
df_submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")
df_submission

In [ ]:
df_submission['Survived'] = 0
df_submission

In [ ]:
df_submission.to_csv("submission1.csv", index =False)

## 2.1 EDA

In [ ]:
len(df_train[df_train["Survived"] == 0]),len(df_train[df_train["Survived"] == 1]) 

In [ ]:
df_train["Survived"].value_counts()

In [ ]:
import seaborn as sns
sns.countplot(x="Survived", data=df_train)

## 2.2 EDA2

In [ ]:
sns.barplot(x="Sex", y="Survived", data=df_train)

In [ ]:
sns.barplot(x="Pclass", y="Survived", data=df_train)

## 2.3 features

In [ ]:
df_train.isnull().sum() #null값있는지 확인

In [ ]:
df_train.info() #Age가 전체 891개 데이터중에 714개가 있으니, null값 확인 Cabin도 204개로 null개수 확인 가능

In [ ]:
df_train

In [ ]:
input_features = [col for col in df_train]

In [ ]:
input_features

In [ ]:
input_features = [col for col in df_train] # 쓸 feature
output_features = ["Survived"]

useless_features = ["PassengerId", "Survived", "Name", "Ticket","Cabin"]

for f in useless_features:
    input_features.remove(f)

X, y = df_train[input_features], df_train[output_features]

In [ ]:
X # feature가 삭제된 것을 알 수 있다.

Sex, Embarked는 숫자로 안되어있기때문에 해당하는 column값의 종류를 숫자로 바꾼다.

In [ ]:
X["Sex"] = X["Sex"].astype("category").cat.codes
X["Embarked"] = X["Embarked"].astype("category").cat.codes

In [ ]:
X 

## 3 데이터 가공하기

In [ ]:
X.info()

### 3.1

In [ ]:
X = X.fillna(-1)
X

### 3.2

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
len(X_train), len(X_test)

### 4.1 sklearn의 DecisionTreeClassifier를 사용해 모델 트레이닝 시키기

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# 모델 학습
model = DecisionTreeClassifier(max_depth=12)
model.fit(X_train, y_train)

#### 4.2 accuracy출력, confusion matrix 그리기, classification report생성해보기, feature importance뽑아보기

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# y_pred = model.predict(X_test)
# accuracy_score(y_test, y_pred)
# classification_report(y_test, y_pred)
# confusion_matrix(y_test, y_pred)
# sns.heatmap(confusion_matrix(y_test, y_pred), annot=True)

def summarize_classification_result(model, X, y):
    y_pred = model.predict(X)
    print(accuracy_score(y, y_pred))
    print(classification_report(y, y_pred))
    print(confusion_matrix(y, y_pred))
    sns.heatmap(confusion_matrix(y, y_pred), annot=True)
summarize_classification_result(model, X_test, y_test)

In [ ]:
X.columns

In [ ]:
import matplotlib.pyplot as plt
model.feature_importances_ #불순물에 따라서 features importance를 뽑아냄

feature_names = X.columns
plt.bar(x = feature_names, height = model.feature_importances_)

## 4.3 sklearn.tree에 내장된 plot_tree를 이용하여, 생성한 decision tree 모델 plot하기

In [ ]:
from sklearn.tree import plot_tree
# plot_tree(model, feature_names = feature_names)
# plt.show()

# 5 Hyperparameter Tuning

## 5.1. sklearn의 gridsearchCV를 사용하여, 튜닝을 해봅시다. max_leaf_nodes 라는 파라미터 하나에 대해서 여러 시도를 해보세요.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
max_leaf_nodes = range(12, 18)
criterion = ["gini", "entropy"]

params = {"max_leaf_nodes": max_leaf_nodes, "criterion": criterion}

tree_grid = GridSearchCV(DecisionTreeClassifier(), params, cv = 5, n_jobs = -1, verbose = 1, scoring = "accuracy")
tree_grid.fit(X_train, y_train)


In [ ]:
my_model = tree_grid.best_estimator_

In [ ]:
tree_grid.best_params_

## 5.2. 튜닝 된 모델들에 대하여, 4.1, 4.2 에서 했던 모든 과정들을 반복해주세요.

In [ ]:
df_test = pd.read_csv("/kaggle/input/titanic/test.csv")

submission_X = df_test[input_features]
submission_X["Sex"] = submission_X["Sex"].astype("category").cat.codes
submission_X["Embarked"] = submission_X["Embarked"].astype("category").cat.codes
submission_X = submission_X.fillna(-1)
submission_y = my_model.predict(submission_X)

In [ ]:
df_submission = pd.read_csv("/kaggle/input/titanic/gender_submission.csv")

In [ ]:
df_submission["Survived"] = submission_y

## 두번째 모델에 대한 결과를 제출해봅시다.

In [ ]:
df_submission.to_csv("submission2.csv", index = False)

## 6. Ensemble

In [ ]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier()
forest.fit(X_train, y_train)
summarize_classification_result(forest, X_test, y_test)

In [ ]:
# max_leaf_nodes
# criterion hini entropy 건들것

### voting classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
clf1 = DecisionTreeClassifier()
clf2 = RandomForestClassifier()
clf3 = XGBClassifier()
clf4 = LGBMClassifier()
clf5 = LogisticRegression()

In [ ]:
hardvoting = VotingClassifier(estimators = [("tress", clf1), ("forest", clf2), ("xgb", clf3), ("lgbm", clf4), ("lr", clf5)], voting ="hard")
softvoting = VotingClassifier(estimators = [("tress", clf1), ("forest", clf2), ("xgb", clf3), ("lgbm", clf4), ("lr", clf5)], voting ="soft")
                              
hardvoting.fit(X_train, y_train)
softvoting.fit(X_train, y_train)                          

hardvoting
softvoting

In [ ]:
summarize_classification_result(hardvoting, X_test, y_test)

In [ ]:
summarize_classification_result(softvoting, X_test, y_test)